In [45]:
import numpy as np
import gdal
import statsmodels.api as sm 
# import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42

%matplotlib inline
#merged slope raster, accumulation area raster, and watershed raster into one three band raster.
# exported this raster into a text file from OSGeo4W command line:
# gdal2xyz input.tif -band 1 -band 2 -band 3 output.txt. This is a slow operation... 


In [16]:
ds = gdal.Open('S:\\JESSE\\GIS\\AngeloSagehorn\\area_slope_shed.tif')


In [17]:
band1 = ds.GetRasterBand(1)
band2 = ds.GetRasterBand(2)
band3 = ds.GetRasterBand(3)

In [31]:
# turn into 1-d arrays, and only use values > 0 (sometimes no-value data is very large negative number)
array = band1.ReadAsArray().flatten()
array1 = array[array>0]
array2 = band2.ReadAsArray().flatten()
array2 = array2[array>0]
array3 = band3.ReadAsArray().flatten()
array3 = array3[array>0]

In [32]:
elderSlope = array2[array3 == 1]
elderArea = array1[array3 == 1]
foxSlope = array2[array3 == 2]
foxArea = array1[array3 == 2]
hankSlope = array2[array3 == 3]
hankArea = array1[array3 == 3]
drySlope = array2[array3 == 4]
dryArea = array1[array3 == 4]

#Band 1 = area
#Band 2 = slope
# Band 3 = watershed

#watershed key:
#Dry = 4
#Hank = 3
#Elder = 1
#Fox = 2

In [33]:
# elderArea = elderArea + 1 #can't take log of 0
# foxArea = foxArea + 1
# hankArea = hankArea + 1 
# dryArea = dryArea + 1 


In [34]:
#GENERATE LOGARITHMIC BIN SPACING
numBins = 21

area_bins_elder = np.logspace(np.log10(min(elderArea)), np.log10(max(elderArea)), numBins)
area_bins_fox = np.logspace(np.log10(min(hankArea)), np.log10(max(hankArea)), numBins)
area_bins_hank = np.logspace(np.log10(min(hankArea)), np.log10(max(foxArea)), numBins)
area_bins_dry = np.logspace(np.log10(min(hankArea)), np.log10(max(dryArea)), numBins)


#Return the indices_elder of the bins to which each value in input array belongs.
indices_elder = np.digitize(elderArea, area_bins_elder)
indices_fox = np.digitize(foxArea, area_bins_fox)
indices_hank = np.digitize(hankArea, area_bins_hank)
indices_dry = np.digitize(dryArea, area_bins_dry)

#COMPUTE MEAN, MEDIAN AND STANDARD DEVIATION OF SLOPE IN EACH AREA BIN
bin_means_elder = [elderSlope[indices_elder == i].mean() for i in range(1, len(area_bins_elder))]
bin_medians_elder = [np.median(elderSlope[indices_elder == i]) for i in range(1, len(area_bins_elder))]
bin_stds_elder = [elderSlope[indices_elder == i].std() for i in range(1, len(area_bins_elder))]

bin_means_fox = [foxSlope[indices_fox == i].mean() for i in range(1, len(area_bins_fox))]
bin_medians_fox = [np.median(foxSlope[indices_fox == i]) for i in range(1, len(area_bins_fox))]
bin_stds_fox = [foxSlope[indices_fox == i].std() for i in range(1, len(area_bins_fox))]

bin_means_hank = [hankSlope[indices_hank == i].mean() for i in range(1, len(area_bins_hank))]
bin_medians_hank = [np.median(hankSlope[indices_hank == i]) for i in range(1, len(area_bins_hank))]
bin_stds_hank = [hankSlope[indices_hank == i].std() for i in range(1, len(area_bins_hank))]

bin_means_dry = [drySlope[indices_dry == i].mean() for i in range(1, len(area_bins_dry))]
bin_medians_dry = [np.median(drySlope[indices_dry == i]) for i in range(1, len(area_bins_dry))]
bin_stds_dry = [drySlope[indices_dry == i].std() for i in range(1, len(area_bins_dry))]



In [62]:
def kde_fit(array):
    dens = sm.nonparametric.KDEUnivariate(array)
    dens.fit()
    return dens

elder_slope_dist = kde_fit(elderSlope.astype(np.double))
elder_area_dist = kde_fit(elderArea.astype(np.double))

In [64]:
# Create a figure
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='col', sharey='row')


plotError = False

ax3.scatter(.5*(area_bins_elder[1:]+area_bins_elder[:-1]),bin_medians_elder, marker = "o", color = 'r', label="Elder")
ax3.scatter(.5*(area_bins_fox[1:]+area_bins_fox[:-1]),bin_medians_fox, marker = "s", color = 'r', label="Fox")
ax3.scatter(.5*(area_bins_hank[1:]+area_bins_hank[:-1]),bin_medians_hank, marker = "o", color = 'b', label="Hank")
ax3.scatter(.5*(area_bins_dry[1:]+area_bins_dry[:-1]),bin_medians_dry, marker = "s", color = 'b', label="Dry")

if plotError:
    ax3.errorbar(.5*(area_bins_elder[1:]+area_bins_elder[:-1]),bin_medians_elder, yerr=bin_stds_elder, fmt = None, capthick=0, ecolor = 'r')
    ax3.errorbar(.5*(area_bins_fox[1:]+area_bins_fox[:-1]),bin_medians_fox, yerr=bin_stds_fox, fmt = None, capthick=0, ecolor = 'r')
    ax3.errorbar(.5*(area_bins_hank[1:]+area_bins_hank[:-1]),bin_medians_hank, yerr=bin_stds_hank, fmt = None, capthick=0, ecolor = 'b')
    ax3.errorbar(.5*(area_bins_dry[1:]+area_bins_dry[:-1]),bin_medians_dry, yerr=bin_stds_dry, fmt = None, capthick=0, ecolor = 'b')

#ax3.fill_between(.5*(area_bins_elder[1:]+area_bins_elder[:-1]),np.subtract(bin_medians_elder, bin_stds_elder), np.add(bin_medians_elder,bin_stds_elder), color = 'r', alpha=0.3)
#ax3.fill_between(.5*(area_bins_fox[1:]+area_bins_fox[:-1]),np.subtract(bin_medians_fox, bin_stds_fox), np.add(bin_medians_fox,bin_stds_fox), color = 'k', alpha=0.3)
#ax3.fill_between(.5*(area_bins_hank[1:]+area_bins_hank[:-1]),np.subtract(bin_medians_hank, bin_stds_hank), np.add(bin_medians_hank,bin_stds_hank), color = 'b', alpha=0.3)
#ax3.fill_between(.5*(area_bins_dry[1:]+area_bins_dry[:-1]),np.subtract(bin_medians_dry, bin_stds_dry), np.add(bin_medians_dry,bin_stds_dry), color = 'y', alpha=0.3)

ax3.legend(bbox_to_anchor=(1.5, 1))
ax3.set_ylabel("Hillslope angle [ % ]")
ax3.set_xlabel("Upslope contributing area [ m$^2$]")
ax3.set_ylim([5,100])
ax3.set_xlim([1,1000000])
ax3.set_yscale('log')
ax3.set_xscale('log')

import matplotlib.ticker
ax3.yaxis.set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax3.xaxis.set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax3.tick_params('both', length=15, width=1, which='major')
ax3.tick_params('both', length=8, width=1, which='minor')

ax4.plot(elder_slope_dist.density, elder_slope_dist.support, lw=1, color='r');
ax1.plot(elder_slope_dist.support, elder_area_dist.density, lw=1, color='r');
ax1.set_yscale('log')



fig.subplots_adjust(hspace=0)
# fig.subplots_adjust(vspace=0)


# plt.savefig('logS_vs_logA.pdf', transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()

OverflowError: Allocated too many blocks